<a href="https://colab.research.google.com/github/apetresc/rotman-api/blob/master/Day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

import matplotlib.pyplot as plt
import numpy as np

import random
from pprint import pprint

# Making an HTTP request

As discussed in the slides

In [ ]:
r = requests.get("https://pokeapi.co/api/v2/pokemon-species/")
r

In [ ]:
r.reason

In [ ]:
if r.ok:
    pprint(r.json())
else:
    print(r.reason)

In [ ]:
r.headers

# JSON

In [ ]:
raw_json = """
{
    "a": true,
    "b": {
        "c": 1,
        "d": [
            {"x": 1},
            {"y": 2}
        ]
    }
}
"""

In [ ]:
parsed_json = json.loads(raw_json)
parsed_json

In [ ]:
print(parsed_json["a"])
print(parsed_json["b"]["d"][0]["x"])

In [ ]:
json.dumps({"a": 1})

# Visualizing Results

In [ ]:
r = requests.get("https://api.coindesk.com/v1/bpi/historical/close.json?start=2012-01-01&end=2019-09-05")

In [ ]:
r.json()

In [ ]:
%matplotlib inline

x = list(r.json()['bpi'].keys())
y = list(r.json()['bpi'].values())

In [ ]:
y

In [ ]:
%matplotlib inline

plt.plot(x, y)

# Access Token

Go get one of your own from https://github.com/settings/tokens! Don't just steal mine!

In [ ]:
r = requests.get("https://api.github.com/orgs/rubikloud/repos")
[repo['full_name'] for repo in r.json()]

In [ ]:
from requests.auth import HTTPBasicAuth

GITHUB_TOKEN = ""
r = requests.get("https://api.github.com/orgs/rubikloud/repos", auth=HTTPBasicAuth(GITHUB_TOKEN, ""))
if r.ok:
    print([repo['full_name'] for repo in r.json()])
else:
    print(r.status_code, r.reason)

# Pagination

In the result above, we can see that GitHub returned some private repositories, as we expected (try opening them in your browser to check!), but it still only returned a handful of repositories. Where did the rest go?

In [ ]:
print(r.headers)

Notice those `Link` headers. Those are GitHub's way of telling us about later or earlier pages. What happens if we iterate over them?

Well, first we need to parse them...

In [ ]:
def parse_github_pagination(link_header):
    link_header_parts = link_header.split(", ")
    return {
        link_header_part.split("; ")[1][5:-1]: link_header_part.split("; ")[0][1:-1]
        for link_header_part in link_header_parts
    }

parse_github_pagination(r.headers['Link'])

Great. So as long as our response _has_ a `rel="next"` in its `Link` header, we can just keep visiting that page, appending our results together as we go.

In [ ]:
repos = [repo["full_name"] for repo in r.json()]
links = parse_github_pagination(r.headers["Link"])
while "next" in links:
    r = requests.get(links["next"], auth=HTTPBasicAuth(GITHUB_TOKEN, ""))
    links = parse_github_pagination(r.headers["Link"])
    repos += [repo["full_name"] for repo in r.json()]

repos

# Mixing them together

In [ ]:
for repo in repos:
    potential_pokemon_name = repo.split('/')[1]
    r = requests.get("https://pokeapi.co/api/v2/pokemon/%s" % potential_pokemon_name.lower())
    is_pokemon = r.ok
    print("%s: %s" % (potential_pokemon_name, is_pokemon))

# GraphQL

In [ ]:
r = requests.post("https://api.github.com/graphql", auth=HTTPBasicAuth(GITHUB_TOKEN, ""), json={"query": """
query {
  organization(login: "rubikloud") {
    name
    repositories(privacy: PRIVATE, first: 5) {
      edges {
        node {
          name
          pullRequests(last: 5) {
            edges {
              node {
                title
                url
              }
            }
          }
        }
      }
    }
  }
}"""})

In [ ]:
r.status_code

In [ ]:
r.json()